# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import datetime
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

Exloring how the data look like

In [2]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [3]:
print("Unique offers so far: ", portfolio.id.nunique())

Unique offers so far:  10


In [4]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [5]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


Now let's explore how many missing data is present

In [6]:
portfolio.isna().sum()

channels      0
difficulty    0
duration      0
id            0
offer_type    0
reward        0
dtype: int64

In [7]:
transcript.isna().sum()

event     0
person    0
time      0
value     0
dtype: int64

In [8]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

Some of the gender values were set as 'O' and there is many NaN values in the dataset.
This data can be useful for prediction or recoomendadion use cases, so we first need to explore the data and decide if it can be dropped or shoule be marked as non complete.

In [9]:
profile.gender.value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

In [10]:
profile.isna().sum()

age                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64

In [11]:
# How many customers with NaN as gender and income have responded to offers
null_inc_and_gender = profile[profile.gender.isna()]
person_viewed_offer = transcript[transcript.event == 'offer completed']
nan_responders = pd.merge(null_inc_and_gender, person_viewed_offer, how='inner', left_on=['id'], right_on=['person'])
print(nan_responders['id'].nunique())

788


2175 customers have not filled the gender data and 212 marked as 'O'.
788 of them are active customers that have responded to one or more offers.
We definitely should take this data into account.

# Cleaning the data and Feature Preprocessing


In [12]:
print(profile.age.value_counts().head())
profile['memberdays'] = datetime.datetime.today().date() - pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.date
profile['memberdays'] = profile['memberdays'].dt.days
profile['income'] = profile['income'].fillna(0)

profile['gender'] = profile['gender'].fillna('X')
profile['gender'] = profile['gender'].map({'X':0,'O':1, 'M':2, 'F':3})
income_bins = [0, 20000, 35000, 50000, 60000, 70000, 90000, 100000, np.inf]
labels = [0,1,2,3,4,5,6,7]
profile['income'] = pd.cut(profile['income'], bins = income_bins, labels= labels, include_lowest=True)
profile['income'].value_counts()

118    2175
58      408
53      372
51      363
54      359
Name: age, dtype: int64


5    3591
2    2889
3    2659
4    2347
0    2175
6    1188
1    1145
7    1006
Name: income, dtype: int64

In [13]:
portfolio['web'] = portfolio['channels'].apply(lambda x: 1 if 'web' in x else 0)
portfolio['email'] = portfolio['channels'].apply(lambda x: 1 if 'email' in x else 0)
portfolio['mobile'] = portfolio['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
portfolio['social'] = portfolio['channels'].apply(lambda x: 1 if 'social' in x else 0)
    
# apply one hot encoding to offer_type column
offer_type = pd.get_dummies(portfolio['offer_type'])

# drop the channels and offer_type column
portfolio.drop(['channels', 'offer_type'], axis=1, inplace=True)

# combine the portfolio and offer_type dataframe to form a cleaned dataframe
portfolio = pd.concat([portfolio, offer_type], axis=1, sort=False)

Now data look like the following

In [14]:
portfolio.head()

,difficulty,duration,id,reward,web,email,mobile,social,bogo,discount,informational
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,0,1,1,1,1,0,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,1,1,1,1,0,0
2,0,4,3f207df678b143eea3cee63160fa8bed,0,1,1,1,0,0,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,1,1,0,1,0,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,1,1,0,0,0,1,0


In [15]:
profile.head()

,age,became_member_on,gender,id,income,memberdays
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162
1,55,20170715,3,0610b486422d4921ae7d2bf64640c50b,7,1009
2,118,20180712,0,38fe809add3b4fcf9315a9694bb96ff5,0,647
3,75,20170509,3,78afa995795e4d85b5d9ceeca43f5fef,6,1076
4,118,20170804,0,a03223e636434f42ac4c3df47e8bac43,0,989


In [16]:
transcript = transcript[transcript.person != None]
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


Now the data looks good and can be used for predictions.

But we need to create 'Fact Table' that has all information that is needed for prediction wheter customer will respond to an offer.

# Joining the data

To be able to build recoomendation engine we need to merge all datasets and create dataset that will aggregate data each user and what offerts it has responded to.


In [17]:
# extract ids for each offer
transcript['offer_id'] = transcript[transcript.event != 'transaction']['value'].apply(lambda x: 
                                                             dict(x).get('offer id') 
                                                             if dict(x).get('offer id') is not None 
                                                             else dict(x).get('offer_id') )

In [18]:
#transcript['reward'] = transcript['reward'].fillna(0)
transcript.isna().sum()

event            0
person           0
time             0
value            0
offer_id    138953
dtype: int64

In [19]:
transcript.head()

,event,person,time,value,offer_id
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},fafdcd668e3743c1bb461111dcafc2a4
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},4d5c57ea9a6940dd891ad53e9dbe8da0


In [20]:
# Transaction offers does ntot have offer id, so we filter them out next

In [21]:
joined_df = pd.merge(profile, transcript[transcript.event != 'transaction'], how='left', left_on=['id'], right_on=['person'])

In [22]:
joined_df.head()

,age,became_member_on,gender,id,income,memberdays,event,person,time,value,offer_id
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,offer received,68be06ca386d4c31939f3a4f0e3dd783,168.0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5
1,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,offer viewed,68be06ca386d4c31939f3a4f0e3dd783,216.0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5
2,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,offer received,68be06ca386d4c31939f3a4f0e3dd783,336.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7
3,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,offer viewed,68be06ca386d4c31939f3a4f0e3dd783,348.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7
4,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,offer received,68be06ca386d4c31939f3a4f0e3dd783,408.0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},fafdcd668e3743c1bb461111dcafc2a4


In [23]:
joined_df['event'] = joined_df['event'].map({'offer received': 0, 'offer viewed': 1, 'offer completed': 2})
joined_df.head()

,age,became_member_on,gender,id,income,memberdays,event,person,time,value,offer_id
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,0.0,68be06ca386d4c31939f3a4f0e3dd783,168.0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5
1,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,1.0,68be06ca386d4c31939f3a4f0e3dd783,216.0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},2906b810c7d4411798c6938adc9daaa5
2,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,0.0,68be06ca386d4c31939f3a4f0e3dd783,336.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7
3,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,1.0,68be06ca386d4c31939f3a4f0e3dd783,348.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0b1e1539f2cc45b7b9fa7c272da2e1d7
4,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,0.0,68be06ca386d4c31939f3a4f0e3dd783,408.0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},fafdcd668e3743c1bb461111dcafc2a4


In [24]:
# Rename column for ease of joining of dataframes
portfolio.rename({'id':'offer_id'}, inplace=True, axis=1)
portfolio.head()

,difficulty,duration,offer_id,reward,web,email,mobile,social,bogo,discount,informational
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,0,1,1,1,1,0,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,1,1,1,1,0,0
2,0,4,3f207df678b143eea3cee63160fa8bed,0,1,1,1,0,0,0,1
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,1,1,0,1,0,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,1,1,0,0,0,1,0


Now all data can be joined together

In [25]:
df = pd.merge(joined_df, portfolio, how='inner', left_on=['offer_id'], right_on=['offer_id'])
df = df.drop(['person', 'value'], axis=1)
df.head()

,age,became_member_on,gender,id,income,memberdays,event,time,offer_id,difficulty,duration,reward,web,email,mobile,social,bogo,discount,informational
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,0.0,168.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
1,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,1.0,216.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
2,68,20180426,2,e2127556f4f64592b11af22de27a7932,4,724,0.0,0.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
3,68,20180426,2,e2127556f4f64592b11af22de27a7932,4,724,1.0,18.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
4,118,20170925,0,8ec6ce2a7e7949b1bf142def7d0e0586,0,937,0.0,408.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0


In [26]:
df.columns

Index(['age', 'became_member_on', 'gender', 'id', 'income', 'memberdays',
       'event', 'time', 'offer_id', 'difficulty', 'duration', 'reward', 'web',
       'email', 'mobile', 'social', 'bogo', 'discount', 'informational'],
      dtype='object')

# Building recommendation matrix

At the moment data for each user has entries for each offer if it was received, viewed and responded to it.
To be able to give valid recoomendations we leave only last user action on each offer (either viewed, responded or ignored).

In [27]:
df[(df.id == '68be06ca386d4c31939f3a4f0e3dd783') & (df.offer_id == '2906b810c7d4411798c6938adc9daaa5')]

,age,became_member_on,gender,id,income,memberdays,event,time,offer_id,difficulty,duration,reward,web,email,mobile,social,bogo,discount,informational
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,0.0,168.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
1,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,1.0,216.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0


In [28]:
users = df['id'].unique()
offers = df['offer_id'].unique()
recommendation_df = pd.DataFrame(columns=df.columns)

In [29]:
print("Number of known users: ", len(users))
print("Number of created offers: ", len(offers))

Number of known users:  16994
Number of created offers:  10


In [ ]:
# Filter data, and log progress .. 
for i, offer in enumerate(offers):
    for j, user in enumerate(users):
        offer_id_actions = df[(df.id == user) & (df.offer_id == offer)]
        # log progress 
        if j % 5000 == 0:
            print('Processing offer %s for user with index: %s' % (i, j))
            print(offer_id_actions)
        
        if len(offer_id_actions) > 1:
            # user viewed or resonded to offer
            if offer_id_actions[offer_id_actions.event == 2]['event'].empty == False:
                # user has not completed an offer
                recommendation_df = recommendation_df.append(offer_id_actions[offer_id_actions.event == 2])
            elif offer_id_actions[offer_id_actions.event == 1]['event'].empty == False:
                # user only viewed offer
                recommendation_df = recommendation_df.append(offer_id_actions[offer_id_actions.event == 1])
            else:
                # Offer could be de received multiple times but ignored
                #print("Filter length", len())
                #print("No event were found in filtered data\n:", offer_id_actions)
                recommendation_df = recommendation_df.append(offer_id_actions[offer_id_actions.event == 0])
        else:
            # offer has been ignored
            recommendation_df = recommendation_df.append(offer_id_actions[offer_id_actions.event == 0])
recommendation_df.head()

Processing offer 0 for user with index: 0
   age  became_member_on  gender                                id income  \
0  118          20170212       0  68be06ca386d4c31939f3a4f0e3dd783      0   
1  118          20170212       0  68be06ca386d4c31939f3a4f0e3dd783      0   

   memberdays  event   time                          offer_id  difficulty  \
0        1162    0.0  168.0  2906b810c7d4411798c6938adc9daaa5          10   
1        1162    1.0  216.0  2906b810c7d4411798c6938adc9daaa5          10   

   duration  reward  web  email  mobile  social  bogo  discount  informational  
0         7       2    1      1       1       0     0         1              0  
1         7       2    1      1       1       0     0         1              0  
Processing offer 0 for user with index: 5000
       age  became_member_on  gender                                id income  \
12512   54          20170729       2  a295311c110841fab8833322d28f76c1      3   
12513   54          20170729       2  a29531

Processing offer 3 for user with index: 0
       age  became_member_on  gender                                id income  \
49759  118          20170212       0  68be06ca386d4c31939f3a4f0e3dd783      0   
49760  118          20170212       0  68be06ca386d4c31939f3a4f0e3dd783      0   
49761  118          20170212       0  68be06ca386d4c31939f3a4f0e3dd783      0   

       memberdays  event   time                          offer_id  difficulty  \
49759        1162    0.0  504.0  2298d6c36e964ae4a3e7e9706d1fb8c2           7   
49760        1162    1.0  504.0  2298d6c36e964ae4a3e7e9706d1fb8c2           7   
49761        1162    2.0  552.0  2298d6c36e964ae4a3e7e9706d1fb8c2           7   

       duration  reward  web  email  mobile  social  bogo  discount  \
49759         7       3    1      1       1       1     0         1   
49760         7       3    1      1       1       1     0         1   
49761         7       3    1      1       1       1     0         1   

       informational  


Processing offer 6 for user with index: 15000
Empty DataFrame
Columns: [age, became_member_on, gender, id, income, memberdays, event, time, offer_id, difficulty, duration, reward, web, email, mobile, social, bogo, discount, informational]
Index: []
Processing offer 7 for user with index: 0
Empty DataFrame
Columns: [age, became_member_on, gender, id, income, memberdays, event, time, offer_id, difficulty, duration, reward, web, email, mobile, social, bogo, discount, informational]
Index: []
Processing offer 7 for user with index: 5000
Empty DataFrame
Columns: [age, became_member_on, gender, id, income, memberdays, event, time, offer_id, difficulty, duration, reward, web, email, mobile, social, bogo, discount, informational]
Index: []
Processing offer 7 for user with index: 10000
Empty DataFrame
Columns: [age, became_member_on, gender, id, income, memberdays, event, time, offer_id, difficulty, duration, reward, web, email, mobile, social, bogo, discount, informational]
Index: []
Processin

In [ ]:
recommendation_df.to_csv('userdata.csv', index=False)

Next step will describe the model and showcasae actual training on prepared data.

In [34]:
# Simple insight on how many times each person responed to a particular offer
gr = df.groupby(['id','offer_id'])#.count()

In [43]:
user_actions = pd.concat([gr.tail(1)]).reset_index(drop=True)
  # .sort_values('id')
  # .reset_index(drop=True))

In [44]:
len(user_actions)

63288

In [45]:
user_actions.head()

,age,became_member_on,gender,id,income,memberdays,event,time,offer_id,difficulty,duration,reward,web,email,mobile,social,bogo,discount,informational
0,118,20170212,0,68be06ca386d4c31939f3a4f0e3dd783,0,1162,1.0,216.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
1,68,20180426,2,e2127556f4f64592b11af22de27a7932,4,724,1.0,18.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
2,118,20170925,0,8ec6ce2a7e7949b1bf142def7d0e0586,0,937,1.0,462.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
3,118,20171002,0,68617ca6246f4fbc85e91a2a49552598,0,930,1.0,432.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
4,65,20180209,2,389bc3fa690240e798340f5a15918d5c,3,800,2.0,498.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0


In [46]:
user_actions.to_csv('useractions.csv', index=False)

In [48]:
df[df.id == 'e12aeaf2d47d42479ea1c4ac3d8286c6' ]

,age,became_member_on,gender,id,income,memberdays,event,time,offer_id,difficulty,duration,reward,web,email,mobile,social,bogo,discount,informational
11,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,168.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
12,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,576.0,2906b810c7d4411798c6938adc9daaa5,10,7,2,1,1,1,0,0,1,0
15776,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,1,1,0,0,0,1,0
15777,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,336.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,1,1,0,0,0,1,0
15778,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,2.0,456.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,1,1,0,0,0,1,0
15779,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,504.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10,5,1,1,0,0,0,1,0
112178,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,0.0,408.0,ae264e3637204a6fb9bb56bc8210ddfd,10,7,10,0,1,1,1,1,0,0
112179,26,20140213,2,e12aeaf2d47d42479ea1c4ac3d8286c6,2,2257,1.0,414.0,ae264e3637204a6fb9bb56bc8210ddfd,10,7,10,0,1,1,1,1,0,0
